# 神经网络
神经网络可以使用 `torch.nn`这个包来构建

前面我们已经了解过 `autograd` 包。 `nn` 包依赖于`autograd` 来定义模型和对其求导。一个 `nn.Module`包含层(layers)，和一个方法`forward(input)`返回`output`.

一个典型的神经网络训练过程包含以下步骤：
* 定义一个包含一些科学系的参数（权重）的神经网络
* 遍历输入的数据集
* 计算损失（结果到正确值之间的距离）
* 将修正值传回网络参数(Propagate gradients back into the network's parameters)
* 更新权重，典型的方式是一种简单的更新规则: `weight = weight - learning_rate * gradient `


## 定义一个网络


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation : y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84) # f5 第五层
        self.fc3 = nn.Linear(84, 10) # f6第六层
        
    def forward(self, x):
        # Max pooling over (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


刚才我们只定义了 `forward` 函数，而 `backward` 函数(计算梯度的地方)是由`autograd`自动定义的。在`forward`里面我们可以使用任何的 Tensor operation。

可学习的参数由 `net.parameters()` 返回

In [15]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


尝试一下 32x32 的输入。注意：这个 LeNet的输入是 32x32。如果要在 MNIST数据集上使用这个网络，需要将图片 resize 到 32 x 32。

In [16]:
input = torch.randn(1, 1, 32,32)
out = net(input)
print(out)

tensor([[ 0.0270, -0.0062, -0.1134,  0.0311,  0.0040, -0.0035, -0.0098,  0.0200,
          0.0368,  0.1061]], grad_fn=<AddmmBackward>)


将梯度内存里的所有参数清零，并反向传播随机的梯度(Zero the gradient of all parameters and backprops with random gradients):

In [ ]:
net.zero_grad()
out.backward(torch,randn(1, 10))

> `torch.nn` 只支持mini-batches. 整个 `torch.nn`包只支持输入是样本的`mini-batch`，并且不是单样本的。
>
> 比如，`nn.Conv2d` 将输入一个4维的 Tensor，四个维度是 `nSamples x nChannels x Height x Width`
>
> 如果你只有一个单变量，就使用 `input.unsqueeze(0)` 来添加一个假的batch dimiension

在往后继续学习之前，我们概括一下目前学过的。
### 概括
* `torch.Tensor` 一个多维数组支持自动求导操作比如 `backward()`. 同时holds the gradient w.r.t the tensor.
* `nn.Module` 神经网络模块，可以很方便的封装参数，同时有很好的迁移到GPU、导出、加载等的组件。
* `nn.Parameter`一种张量，当作为一个`Module`的属性的时候会自动被看作是一种参数
* `autograd.Function` 实现自动分级操作的前向和后向定义。 每个Tensor操作都会创建至少一个Function节点，该节点连接到创建Tensor并对其历史进行编码的函数。

### 截止目前我们学习了

* 定义一个神经网络
* 处理了·输入和反向传播

### 尚未学习的
* 计算损失
* 更新网络的权重

## 损失函数